In [ ]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-1106")

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(
        description="Stock symbol of the company.Example: AAPL,TSLA",
    )


class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatement"
    description = """
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()["annualReports"]


class CompanyStockPerformanceTool(BaseTool):
    name = "CompanyStockPerformance"
    description = """
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ],
)

prompt = "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."

agent.invoke(prompt)

In [ ]:
# SQLatabaseToolkit
from langchain.agents import create_sql_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-4-1106-preview",
)
db = SQLDatabase.from_uri("sqlite:///movies.sqlite")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

agent.invoke(
    "Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors also include their gross revenue."
)

In [9]:
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper, WikipediaAPIWrapper
from langchain.agents.agent_toolkits import FileManagementToolkit
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1,)

class SearchToolArgsSchema(BaseModel):
  query: str = Field(
    description="The query you will search for.Example query: Research about the XZ backdoor"
  )


class WikipediaSearchTool(BaseTool):
  name = "WikipediaSearchTool"
  description = "Use this tool to find the website for the given query."
  args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema
  
  def _run(self, query):
    wiki = WikipediaAPIWrapper()
    return wiki.run(query)
  

class DuckDuckGoSearchTool(BaseTool):
  name = "DuckDuckGoSearchTool"
  description = "Use this tool to find the website for the given query."
  args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

  def _run(self, query):
    ddg = DuckDuckGoSearchAPIWrapper()
    return ddg.run(query)
  

class ToTxtTool(BaseTool):
  name = "ToTxtTool"
  description = "Converts the output to a text file"
  
  def _run(self, output):
    tools = FileManagementToolkit(
      root_dir=str(".txt/"),
      selected_tools=["read_file", "write_file", "list_directory"],
    ).get_tools()
    read_tool, write_tool, list_tool = tools
    return write_tool.invoke({"file_path": "output.txt", "text": output})


agent = initialize_agent(
  llm=llm,
  verbose=True,
  agent=AgentType.OPENAI_FUNCTIONS,
  handle_parsing_errors=True,
  tools=[
    WikipediaSearchTool(),
    DuckDuckGoSearchTool(),
    ToTxtTool()
  ],
)


prompt = PromptTemplate.from_template(
  "Search the {query}, go to the related website, extract the content with text, and save it as a txt file."
)

chain = ({"query": RunnablePassthrough()} | prompt | agent)

chain.invoke("Research about the XZ backdoor")



> Entering new AgentExecutor chain...

Invoking: `WikipediaSearchTool` with `{'query': 'Research about the XZ backdoor'}`


Page: XZ Utils backdoor
Summary: On 29 March 2024, software developer Andres Freund reported that he had found a maliciously introduced backdoor in the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 released in February 2024.While xz is commonly present in most Linux distributions, the backdoor only targeted Debian- and RPM-based systems running on the x86-64 architecture. At the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions.The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux systems. The issue has been assigned a CVSS score of 10.0, the highest possible score.



Page: Supply chain attack
Summary: A supply chain attack is a cyber-attack that seeks

{'input': StringPromptValue(text='Search the Research about the XZ backdoor, go to the related website, extract the content with text, and save it as a txt file.'),
 'output': 'I have extracted the information about the XZ backdoor and saved it as a text file. You can download the file [here](output.txt).'}